In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
import mord
from collections import Counter

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['cheerful_last'] = mood.groupby('user_id')['cheerful'].shift()
mood.loc[(pd.isnull(mood.cheerful_last)), 'cheerful_last'] = mood['cheerful']
mood_cheerful = mood.drop(["anxious", "content", "bored", "user_id", "response_time"], axis=1)

In [3]:
change_list = []
for index, row in mood_cheerful.iterrows():
    if row['cheerful'] == row['cheerful_last']:
        change_list.append(1)
    if row['cheerful'] > row['cheerful_last']:
        change_list.append(2)
    if row['cheerful'] < row['cheerful_last']:
        change_list.append(0)

mood_cheerful['change'] = change_list

mood_cheerful = mood_cheerful.drop(['cheerful', 'cheerful_last'], axis=1)

In [4]:
# Normalize predictors
mood_normz = mood_cheerful.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [5]:
def macroaverage_mae(test, pred):
    mae_1, mae_2, mae_3 = (0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_3 += (abs(test[i]-pred[i]))

    cnt_1, cnt_2, cnt_3 = (0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_1 += 1
        if test[z] == 1:
            cnt_2 += 1
        if test[z] == 2:
            cnt_3 += 1

    mae_macroaverage = ((mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3)) / 6
    return mae_macroaverage 

In [6]:
# X = predictors, y = target
X = mood_normz.iloc[:,:-1]
y = mood_normz.iloc[:, -1:]

# Convert float to int
y = y.astype(int)

In [7]:
X_train, y_train = shuffle(X, y)

In [8]:
# Convert oversample dataset to Pandas DataFrame
X = pd.DataFrame(data=X_train)
y = pd.DataFrame(data=y_train)

In [9]:
n_split=10
f1_scores = []
f1_scores_classes = []
mmae_scores = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    
    pred = []
    for a in range(len(y_test)):
        pred.append(1)
    
    test = y_test.values
    
    f1_scores_classes.append(f1_score(test, pred, average=None))
    f1_scores.append(round(f1_score(test, pred, average='weighted'), 4))
    mmae_scores.append(macroaverage_mae(test, pred))

/Users/Stijn/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
print("Mean F1-score per class: ", [round(np.mean(x), 3) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 3) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.0, 0.759, 0.0]
STD F1-score per class:  [0.0, 0.009, 0.0]


In [11]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 3))
print("Std MA F1-scores: ", round(np.std(f1_scores), 3))

Macro-averaged F1-scores:  [0.4548, 0.4702, 0.4544, 0.4684, 0.4609, 0.4928, 0.4834, 0.4609, 0.4646, 0.4388]
Mean MA F1-scores:  0.465
Std MA F1-scores:  0.014


In [12]:
mmae_scores_1 = []
for a in mmae_scores:
    for b in a:
        mmae_scores_1.append(round(b, 3))
print("Macro-averaged MAE: ", mmae_scores_1)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 3))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 3))

Macro-averaged MAE:  [0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333, 0.333]
Mean MMAE-scores:  0.333
Std MMAE-scores:  0.0
